In [1]:
import pandas as pd
import os

In [2]:
companies = {
    # Nông nghiệp
    "Hoàng Anh Gia Lai": "HAG",
    "Tập đoàn PAN": "PAN",
    "Dabaco Việt Nam": "DBC",
    "Giống cây trồng Trung Ương": "NSC",
    "Mía đường Lam Sơn": "LSS",
    "Mía đường Thành Thành Công Tây Ninh": "SBT",
    "Mía đường Sơn La": "SLS",
    "Mía đường Sóc Trăng": "SLS",
    "Thực phẩm Sao Ta": "FMC",
    "Nông nghiệp Quốc tế Hoàng Anh Gia Lai": "HNG",
    "Thực phẩm Cholimex": "CMF",
    "Giống cây trồng Miền Nam": "SSC",
    "Công ty cổ phần Vĩnh Hoàn": "VHC",
    "Công ty cổ phần Nam Việt": "ANV",
    "Công ty cổ phần Thủy sản Minh Phú": "MPC",
    "Công ty cổ phần Thực phẩm Bích Chi": "BFC",
    "Công ty cổ phần Tập đoàn Lộc Trời": "LTG",
    "CTCP Nông nghiệp BaF Việt Nam": "BAF",
    "CTCP Dược phẩm Cửu Long": "DCL",
    "CTCP Thuốc sát trùng Việt Nam": "VFG",
    "CTCP XNK Thủy sản Bến Tre": "ABT",
    "CTCP XNK An Giang": "AGM",
    "CTCP Chế biến Gỗ Đức Thành": "GDT",
    "CTCP Tập đoàn Dược Bảo Châu": "BCP",
    "CTCP Tập đoàn Sao Mai": "ASM",
    "CTCP Dầu thực vật Tường An": "TAC",
    "CTCP Dầu thực vật Cái Lân": "VOC"
}

In [3]:
# Khởi tạo list chứa dữ liệu từng công ty
all_data = []

# Đọc từng file và gắn mã công ty
for full_name, ticker in companies.items():
    file_path = f"pre-processing/{full_name}_data.csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)

        # Thêm mã công ty
        df["company_id"] = ticker

        # Chuyển cột thời gian nếu có
        if "quarter" in df.columns:
            df["quarter"] = pd.to_datetime(df["quarter"])

        all_data.append(df)
    else:
        print(f"⚠️ File không tồn tại: {file_path}")

# Gộp toàn bộ dữ liệu
df_all = pd.concat(all_data, ignore_index=True)

# Sắp xếp theo thời gian và công ty
df_all = df_all.sort_values(["company_id", "quarter"])

# Tạo biến mục tiêu P(t+1)
df_all["P_t_plus_1"] = df_all.groupby("company_id")["P"].shift(-1)

# Xoá dòng thiếu dữ liệu đầu vào
df_model = df_all.dropna(subset=["P_t_plus_1", "P/E", "EPS", "D/E", "SIZE"])

df_model.set_index(['company_id', 'quarter'], inplace=True)

df_model

C:\Users\hype1\AppData\Local\Temp\ipykernel_36660\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_36660\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_36660\3346459644.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["quarter"] = pd.to_datetime(df["quarter"])
C:\Users\hype1\AppData\Local\Temp\ipykernel_36660\3346459644.py:15: UserWarning: Could not infer for

P       EPS       D/E           P/E       SIZE  \
company_id quarter                                                              
ABT        2019-01-01  66.691058  0.079522  0.293322    838.652734  26.991506   
           2019-04-01  70.844991  0.111456  0.261232    635.633188  26.991072   
           2019-07-01  76.766997  0.132993  0.284692    577.223709  26.998222   
           2019-10-01  75.971624  0.024478  0.303334   3103.658648  27.011982   
           2020-01-01  76.001590  0.005053  0.225539  15039.625760  26.923983   
...                          ...       ...       ...           ...        ...   
VOC        2023-07-01   6.983779  0.010428  0.077558    669.713203  28.631544   
           2023-10-01   6.903968  0.086811  0.025926     79.528982  28.473876   
           2024-01-01   5.685511  0.005660  0.024877   1004.426007  28.475937   
           2024-04-01   4.880346  0.010070  0.028082    484.660716  28.415581   
           2024-07-01   4.471948  0.007773  0.030446    575.328069  28.421724   

                       P_t_plus_1  
company_id quarter                 
ABT        2019-01-01   70.844991  
           2019-04-01   76.766997  
           2019-07-01   75.971624  
           2019-10-01   76.001590  
           2020-01-01   82.677715  
...                           ...  
VOC        2023-07-01    6.903968  
           2023-10-01    5.685511  
           2024-01-01    4.880346  
           2024-04-01    4.471948  
           2024-07-01    4.662467  

[410 rows x 6 columns]

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Giả sử df_model là DataFrame chứa dữ liệu của bạn
# Khởi tạo StandardScaler
scaler = StandardScaler()

# Chuẩn hóa toàn bộ DataFrame
df_model_scaled = df_model.copy()  # Tạo bản sao để không thay đổi dữ liệu gốc
df_model_scaled[:] = scaler.fit_transform(df_model)

# Kiểm tra lại dữ liệu đã chuẩn hóa
df_model_scaled


P       EPS       D/E       P/E      SIZE  \
company_id quarter                                                        
ABT        2019-01-01 -0.457286 -0.192553 -0.134405 -0.114135 -1.664964   
           2019-04-01 -0.456975 -0.069536 -0.137919 -0.114183 -1.665339   
           2019-07-01 -0.456531  0.013432 -0.135350 -0.114196 -1.659180   
           2019-10-01 -0.456591 -0.404592 -0.133309 -0.113605 -1.647328   
           2020-01-01 -0.456588 -0.479420 -0.141828 -0.110810 -1.723123   
...                         ...       ...       ...       ...       ...   
VOC        2023-07-01 -0.461760 -0.458716 -0.158033 -0.114175 -0.252373   
           2023-10-01 -0.461766 -0.164474 -0.163688 -0.114313 -0.388175   
           2024-01-01 -0.461858 -0.477081 -0.163803 -0.114096 -0.386400   
           2024-04-01 -0.461918 -0.460096 -0.163452 -0.114218 -0.438385   
           2024-07-01 -0.461948 -0.468944 -0.163193 -0.114197 -0.433095   

                       P_t_plus_1  
company_id quarter                 
ABT        2019-01-01   -0.457442  
           2019-04-01   -0.457004  
           2019-07-01   -0.457063  
           2019-10-01   -0.457060  
           2020-01-01   -0.456567  
...                           ...  
VOC        2023-07-01   -0.462169  
           2023-10-01   -0.462259  
           2024-01-01   -0.462318  
           2024-04-01   -0.462349  
           2024-07-01   -0.462335  

[410 rows x 6 columns]

In [5]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Pooled OLS
pooled_model = PooledOLS(y, X)
pooled_results = pooled_model.fit(cov_type="robust")

# In kết quả ước lượng
pooled_results

Dep. Variable:,P_t_plus_1,R-squared:,0.0700
Estimator:,PooledOLS,R-squared (Between):,0.0721
No. Observations:,410,R-squared (Within):,-0.5087
Date:,"Fri, May 09 2025",R-squared (Overall):,0.0700
Time:,08:56:03,Log-likelihood,-566.89
Cov. Estimator:,Robust,,
,,F-statistic:,7.6185
Entities:,24,P-value,0.0000
Avg Obs:,17.083,Distribution:,"F(4,405)"
Min Obs:,1.0000,,
Max Obs:,42.000,F-statistic (robust):,8.2068


In [6]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Fixed Effects (FEM)
fixed_effects_model = PanelOLS(y, X, entity_effects=True)  # entity_effects=True để có hiệu ứng cố định theo công ty
fixed_effects_results = fixed_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
fixed_effects_results


Dep. Variable:,P_t_plus_1,R-squared:,0.0083
Estimator:,PanelOLS,R-squared (Between):,-0.1233
No. Observations:,410,R-squared (Within):,0.0083
Date:,"Fri, May 09 2025",R-squared (Overall):,-0.0302
Time:,08:56:03,Log-likelihood,-6.0906
Cov. Estimator:,Robust,,
,,F-statistic:,0.7945
Entities:,24,P-value,0.5293
Avg Obs:,17.083,Distribution:,"F(4,382)"
Min Obs:,1.0000,,
Max Obs:,42.000,F-statistic (robust):,1.4559


In [7]:
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

# Thiết lập biến phụ thuộc và biến độc lập
y = df_model_scaled['P_t_plus_1']  # Biến mục tiêu (giá cổ phiếu t+1)
X = df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']]  # Các biến độc lập

# Thêm hằng số vào mô hình (intercept)
X = sm.add_constant(X)

# Áp dụng mô hình Random Effects
random_effects_model = RandomEffects(y, X)
random_effects_results = random_effects_model.fit(cov_type="robust")

# In kết quả ước lượng
random_effects_results


Dep. Variable:,P_t_plus_1,R-squared:,0.0088
Estimator:,RandomEffects,R-squared (Between):,-0.0327
No. Observations:,410,R-squared (Within):,0.0076
Date:,"Fri, May 09 2025",R-squared (Overall):,-0.1176
Time:,08:56:03,Log-likelihood,-21.018
Cov. Estimator:,Robust,,
,,F-statistic:,0.8972
Entities:,24,P-value,0.4655
Avg Obs:,17.083,Distribution:,"F(4,405)"
Min Obs:,1.0000,,
Max Obs:,42.000,F-statistic (robust):,1.0588


In [8]:
fixed_effects_results.f_statistic

Model F-statistic (homoskedastic)
H0: All parameters ex. constant are zero
Statistic: 0.7945
P-value: 0.5293
Distributed: F(4,382)
WaldTestStatistic, id: 0x1dd80038f20

In [9]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.stats.stattools import durbin_watson
import scipy.linalg as la

# Lấy các hệ số ước lượng và phương sai-covariance của các mô hình
beta_FE = fixed_effects_results.params
beta_RE = random_effects_results.params
var_FE = fixed_effects_results.cov
var_RE = random_effects_results.cov

# Tính toán kiểm định Hausman
diff = beta_FE - beta_RE
diff_cov = var_FE - var_RE

# Kiểm tra tính khả thi của ma trận khác biệt và tính toán giá trị kiểm định Hausman
hausman_stat = diff.T @ la.inv(diff_cov) @ diff
df = len(beta_FE)  # số bậc tự do (số biến độc lập)

# P-value sử dụng phân phối Chi-squared
from scipy.stats import chi2
p_value = 1 - chi2.cdf(hausman_stat, df)

print(f"Hausman Statistic: {hausman_stat}")
print(f"P-value: {p_value}")

Hausman Statistic: -0.5798319381346186
P-value: 1.0


In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Thêm hằng số vào các biến độc lập
X = add_constant(df_model_scaled[['P/E', 'EPS', 'D/E', 'SIZE']])

# Tính VIF cho từng biến độc lập
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# In kết quả
print(vif_data)

  Variable       VIF
0    const  1.000000
1      P/E  1.006385
2      EPS  1.064264
3      D/E  1.009574
4     SIZE  1.058658


In [11]:
from statsmodels.stats.stattools import durbin_watson

# Kiểm tra tự tương quan cho mô hình Pooled OLS
pooled_resids = pooled_results.resids
print(f'Durbin-Watson statistic for Pooled OLS: {durbin_watson(pooled_resids)}')

# Kiểm tra tự tương quan cho mô hình FEM
fixed_effects_resids = fixed_effects_results.resids
print(f'Durbin-Watson statistic for FEM: {durbin_watson(fixed_effects_resids)}')

# Kiểm tra tự tương quan cho mô hình REM
random_effects_resids = random_effects_results.resids
print(f'Durbin-Watson statistic for REM: {durbin_watson(random_effects_resids)}')


Durbin-Watson statistic for Pooled OLS: 0.325197227157989
Durbin-Watson statistic for FEM: 0.7448112185741129
Durbin-Watson statistic for REM: 0.7464380448294001


In [12]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_bp_test = het_breuschpagan(pooled_resids, X)

print("\n📊 Breusch-Pagan test for Pooled OLS:")
for label, value in zip(labels, pooled_bp_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_bp_test = het_breuschpagan(fixed_effects_resids, X)

print("\n📊 Breusch-Pagan test for FEM:")
for label, value in zip(labels, fixed_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_bp_test = het_breuschpagan(random_effects_resids, X)

print("\n📊 Breusch-Pagan test for REM:")
for label, value in zip(labels, random_effects_bp_test):
    print(f"{label:<25}: {value:.4f}")


📊 Breusch-Pagan test for Pooled OLS:
LM Statistic             : 32.8138
LM P-value               : 0.0000
F-statistic              : 8.8084
F-statistic P-value      : 0.0000

📊 Breusch-Pagan test for FEM:
LM Statistic             : 11.8659
LM P-value               : 0.0184
F-statistic              : 3.0176
F-statistic P-value      : 0.0179

📊 Breusch-Pagan test for REM:
LM Statistic             : 8.8386
LM P-value               : 0.0653
F-statistic              : 2.2308
F-statistic P-value      : 0.0650


In [13]:
from statsmodels.stats.diagnostic import het_white

# Format kết quả kiểm định
labels = ['LM Statistic', 'LM P-value', 'F-statistic', 'F-statistic P-value']

# Kiểm tra phương sai của sai số thay đổi cho mô hình Pooled OLS
pooled_white_test = het_white(pooled_resids, X)

print("\n📊 White test for Pooled OLS:")
for label, value in zip(labels, pooled_white_test):
    print(f"{label:<25}: {value:.4f}")

## Kiểm tra phương sai của sai số thay đổi cho mô hình FEM
fixed_effects_white_test = het_white(fixed_effects_resids, X)

print("\n📊 White test for FEM:")
for label, value in zip(labels, fixed_effects_white_test):
    print(f"{label:<25}: {value:.4f}")

# Kiểm tra phương sai của sai số thay đổi cho mô hình REM
random_effects_white_test = het_white(random_effects_resids, X)

print("\n📊 White test for REM:")
for label, value in zip(labels, random_effects_white_test):
    print(f"{label:<25}: {value:.4f}")



📊 White test for Pooled OLS:
LM Statistic             : 302.3043
LM P-value               : 0.0000
F-statistic              : 79.1982
F-statistic P-value      : 0.0000

📊 White test for FEM:
LM Statistic             : 168.7222
LM P-value               : 0.0000
F-statistic              : 19.7298
F-statistic P-value      : 0.0000

📊 White test for REM:
LM Statistic             : 147.7185
LM P-value               : 0.0000
F-statistic              : 15.8905
F-statistic P-value      : 0.0000
